In [ ]:
%%time
import json
import inspect
import tqdm
import contractions
from collections import Counter

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.nlp_extraction import *


In [ ]:
%%time
user = "Nostranomist"
# user = "CryptoKaleo"
with open("../data/tweets/u"+user+"tweets.json", "r") as r:
    tweets = json.load(r)
tweet_text = []
for ind, tweet in enumerate(tweets):
    sent = tweet["full_text"]
    
    try: #remove tweets of only tagging someone
        while sent[0] == "@":
            sent = sent.split(" ", 1)[1]
        sent = contractions.fix(sent.replace("&amp;", "and").replace("@", ""))
        tweet_text.append((sent, {"created_at":tweet["created_at"], "id" :tweet["id"]}))
    except:
        tweets.pop(ind)

In [ ]:
len(tweets)

In [ ]:
%%time
docs = []
all_tickers = []
# need to disable pipes
with open("senti.txt", "w+") as w:
    for doc, context in nlp.pipe(tweet_text, as_tuples = True, n_process=-1):
        
        if doc._.tickers:
            all_tickers.extend(doc._.tickers)

            w.write("***"+str(ind)+" "+" ".join(doc._.tickers)+"***"+str(doc._.polarity)+" "+str(doc._.subjectivity)+"\n")
            w.write(doc.text+"\n\n")
        doc._.tweet_id = context["id"]
        doc._.tweeted_at = context["created_at"]
        docs.append(doc)

In [ ]:
docs[0]._.polarity

In [ ]:
with open("specific.txt", "w+") as w:
    for doc in docs:
        if "ENJ" in doc._.tickers:
            w.write(str(doc._.polarity)+"\n"+doc._.tweeted_at+"\n"+doc.text+" "+"\n\n")

In [ ]:
Counter(all_tickers).most_common()